In [1]:
import numpy as np


In [3]:

class RBM:
    def __init__(self, num_visible, num_hidden, learning_rate=0.1, epochs=100, batch_size=10):
        self.num_visible = num_visible
        self.num_hidden = num_hidden
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size

        self.weights = np.random.randn(num_visible, num_hidden) * 0.1
        self.visible_bias = np.zeros(num_visible)
        self.hidden_bias = np.zeros(num_hidden)

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def gibbs_sampling(self, visible_prob):
        hidden_prob = self.sigmoid(np.dot(visible_prob, self.weights) + self.hidden_bias)
        hidden_state = np.random.binomial(1, hidden_prob)

        visible_recon_prob = self.sigmoid(np.dot(hidden_state, self.weights.T) + self.visible_bias)
        visible_recon = np.random.binomial(1, visible_recon_prob)

        return hidden_state, visible_recon

    def contrastive_divergence(self, input_data):
        num_samples = input_data.shape[0]

        positive_hidden_prob = self.sigmoid(np.dot(input_data, self.weights) + self.hidden_bias)
        positive_associations = np.dot(input_data.T, positive_hidden_prob)

        hidden_state = np.random.binomial(1, positive_hidden_prob)
        negative_visible_prob = self.sigmoid(np.dot(hidden_state, self.weights.T) + self.visible_bias)
        negative_hidden_prob = self.sigmoid(np.dot(negative_visible_prob, self.weights) + self.hidden_bias)
        negative_associations = np.dot(negative_visible_prob.T, negative_hidden_prob)

        self.weights += self.learning_rate * ((positive_associations - negative_associations) / num_samples)
        self.visible_bias += self.learning_rate * np.mean(input_data - negative_visible_prob, axis=0)
        self.hidden_bias += self.learning_rate * np.mean(positive_hidden_prob - negative_hidden_prob, axis=0)

    def train(self, data):
        num_batches = len(data) // self.batch_size

        for epoch in range(self.epochs):
            for batch in range(num_batches):
                batch_data = data[batch * self.batch_size : (batch + 1) * self.batch_size]
                self.contrastive_divergence(batch_data)

            reconstruction_error = np.mean(np.abs(data - self.reconstruct(data)))
            print(f"Epoch {epoch + 1}/{self.epochs}, Reconstruction Error: {reconstruction_error}")

    def reconstruct(self, data):
        hidden_prob = self.sigmoid(np.dot(data, self.weights) + self.hidden_bias)
        visible_recon_prob = self.sigmoid(np.dot(hidden_prob, self.weights.T) + self.visible_bias)
        return visible_recon_prob

if __name__ == "__main__":
    num_visible_units = 10
    num_hidden_units = 5
    rbm = RBM(num_visible=num_visible_units, num_hidden=num_hidden_units)

In [5]:

data = np.random.randint(0, 2, size=(100, num_visible_units))


rbm.train(data)

Epoch 1/100, Reconstruction Error: 0.4981227668361333
Epoch 2/100, Reconstruction Error: 0.4968361242861507
Epoch 3/100, Reconstruction Error: 0.4959885428481622
Epoch 4/100, Reconstruction Error: 0.4954310249264029
Epoch 5/100, Reconstruction Error: 0.4949726663994579
Epoch 6/100, Reconstruction Error: 0.4946339936104566
Epoch 7/100, Reconstruction Error: 0.49430932404491773
Epoch 8/100, Reconstruction Error: 0.4938966896058109
Epoch 9/100, Reconstruction Error: 0.4934878739311935
Epoch 10/100, Reconstruction Error: 0.4931551906155756
Epoch 11/100, Reconstruction Error: 0.49271643401318505
Epoch 12/100, Reconstruction Error: 0.4920809915479637
Epoch 13/100, Reconstruction Error: 0.4915218594956124
Epoch 14/100, Reconstruction Error: 0.4908061382005786
Epoch 15/100, Reconstruction Error: 0.49020794667327094
Epoch 16/100, Reconstruction Error: 0.489287593957046
Epoch 17/100, Reconstruction Error: 0.48828871776287247
Epoch 18/100, Reconstruction Error: 0.4872968983435972
Epoch 19/100, Re

In [6]:
# Reconstruct input data
reconstructed_data = rbm.reconstruct(data)
print("Original Data:")
print(data)


Original Data:
[[0 0 1 0 1 1 1 0 0 0]
 [1 0 1 0 1 1 0 1 0 0]
 [1 1 0 0 0 0 1 1 0 0]
 [0 0 1 0 0 1 1 1 0 0]
 [0 1 0 0 1 1 0 0 0 1]
 [1 0 1 1 1 1 0 0 0 0]
 [1 1 0 1 0 1 0 0 0 1]
 [0 0 0 1 0 1 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 0 1 0 1]
 [1 0 1 0 1 1 1 1 1 1]
 [1 0 1 1 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 0 1]
 [0 1 1 1 0 1 0 1 1 1]
 [1 1 1 0 1 0 1 0 1 1]
 [1 0 1 0 0 0 0 0 0 0]
 [1 0 0 0 1 1 0 1 0 0]
 [1 1 1 1 0 1 1 1 0 1]
 [0 1 0 1 0 0 0 1 1 0]
 [0 1 0 0 1 1 1 0 0 1]
 [1 0 1 1 0 0 1 0 1 0]
 [1 0 1 0 0 1 0 1 0 1]
 [1 1 0 1 0 1 1 0 0 0]
 [1 0 0 1 1 0 1 1 1 1]
 [1 0 1 1 1 0 0 0 1 1]
 [1 0 0 1 1 0 1 1 0 0]
 [1 0 1 1 0 1 0 0 0 0]
 [0 1 0 1 0 1 1 1 1 0]
 [1 1 1 0 1 0 0 1 1 1]
 [1 0 0 0 1 1 1 0 0 0]
 [1 1 0 0 1 1 1 0 0 0]
 [1 0 1 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0 1 1]
 [1 1 1 1 1 1 0 0 0 1]
 [1 1 0 1 0 0 0 1 0 1]
 [1 0 1 1 1 1 0 0 1 0]
 [0 1 1 0 0 1 0 0 0 0]
 [0 1 0 1 0 1 1 0 1 0]
 [0 1 0 0 0 0 0 0 0 0]
 [1 1 1 0 1 0 0 0 0 0]
 [1 0 0 1 0 1 0 1 1 1]
 [0 1 1 1 1 1 1 1 0

In [7]:
print("Reconstructed Data:")
print(reconstructed_data)

Reconstructed Data:
[[0.56130468 0.1924447  0.50180003 0.11829976 0.93109454 0.70236712
  0.53427553 0.31239112 0.37535321 0.34866557]
 [0.70651578 0.20886168 0.75222074 0.35707602 0.82975438 0.8313513
  0.1017416  0.42178057 0.26926217 0.37236515]
 [0.78440166 0.85617187 0.05163545 0.4762378  0.08006709 0.35135851
  0.86876588 0.80362584 0.31508594 0.27923283]
 [0.45016059 0.44709795 0.601948   0.29597586 0.36303245 0.48574983
  0.4570665  0.57452615 0.51369088 0.48158282]
 [0.54388357 0.41703033 0.40585497 0.08167409 0.80933826 0.89194087
  0.21903928 0.31509888 0.46346521 0.34107108]
 [0.77086674 0.18464187 0.86040819 0.68676053 0.72569183 0.76153107
  0.06202394 0.54584141 0.20955373 0.41004074]
 [0.88361809 0.73721561 0.34365289 0.58017549 0.09639002 0.84239489
  0.08375054 0.76281178 0.17661597 0.28486423]
 [0.82660876 0.61286746 0.59201993 0.67563983 0.10885891 0.74181354
  0.08340432 0.76410153 0.21506523 0.36803822]
 [0.87908174 0.55957635 0.2133808  0.40548705 0.37619565 0.69